In [ ]:
#180620

#sarahfong

#doing the same analysis as before, but migrating to hg38 alignments downloaded from ENCODE

In [ ]:
#!/usr/bin/python

#sarahfong

#180207 - The MultiIntersect command in step 3 has been relaxed. 
    #Originally, we evaluated only reciprocal overlapping peak fragments, resulting in few and small peaks called among all fragments. 
    #Changed command from multiIntersectBed -i <<files>> -f 0.5 -r to multiIntersectBed -i <<files>> -f 0.5 -e
        #Includes fragment if there is 50% overlap of either fragment. 
    #This accomodates overlapping very large peaks with very small peaks.

#180123 - There are two methods in step 2 for subtracting overlapping H3K27ac and H3K4me3 ChIP-SEQ fragments.

#Stringent requirements for calling an enhancer include 
    #(1) only non-overlapping H3K27ac+ H3K4me3- fragments.

#Relaxed requirements for calling an enhancer include 
    #(1) non-overlapping H3K27ac+ H3K4me3- fragments (-A command) 
    #(2) fragments with 50% overlap of -a and -b in order to subtract one region from another.

    #Note about Relaxed requirements 
        #This method is based off Mary Lauren's analysis histone modifications of enhancer regions in roadmap. 
        #This is the method used in Villar 2015 to call enhancers. 

#180118 - the purpose of this script is to iterate through each ENCODE tissue (e.g. E034, E035, etc.) to find any human enhancers. 
    #an enhancer as H3K27ac+ and H3K4me3-. 
    #H3K4me3+ marks active promoters.


##### Method #####

#step 1-  Glob histone modification files

#step 2-  Iterate through each tissue ID file using bedtools - find non-overlapping ChIP-seq regions and write .bed file

#             Active enhancers: All H3k27ac marks non-overlapping w/ H3k4me3 found in that tissue. I.e. H3k27ac+/H3k4me3-

#step 3-  I will then multiintersect the resultant enhancer files for all tissues. This will be run with the script "roadmap_find_common_enhancer.py

In [2]:
# import modules

import os
import sys
import glob
import datetime
import pandas

now = str(datetime.date.today())
print(now)

2018-06-20


In [3]:
# hg38path
path = "/dors/capra_lab/data_clean/roadmap_epigenomics/hg38/stable_replicated_peaks/"

os.chdir(path)

In [4]:
#step 1- find matching H3k27ac and H3k4me3 datasets per tissue_id

h3k27ac = glob.glob("*H3K27ac*.bed")
h3k4me3 = glob.glob("*H3K4me3*.bed")

h3k27ac_dict = dict((name.split("_")[2], name) for name in h3k27ac) 
h3k4me3_dict = dict((name.split("_")[2], name) for name in h3k4me3)

matched_dataset=list(key for key in h3k27ac_dict.keys() if key in h3k4me3_dict.keys())

tissue_dict ={}
for biosample in matched_dataset:
    if biosample in h3k27ac_dict.keys():

        tissue_id = "".join(h3k27ac_dict[biosample].split('_')[3:-1])
        tissue_dict[biosample] = tissue_id
        

In [38]:
print(tissue_dict.values())

dict_values(['H9', 'chorionicvillus', 'liver', 'CD4-positive,alpha-betamemoryTcell', 'musclelayerofduodenum', 'kidney', 'HUES6', 'CD4-positive,CD25-positive,alpha-betaregulatoryTcell', 'chorion', 'trophoblastcell', 'angulargyrus', 'foreskinmelanocyte', 'smallintestine', 'heartrightventricle', 'rightcardiacatrium', 'adrenalgland', 'iPS-18a', 'mesendoderm', 'foreskinfibroblast', 'naivethymus-derivedCD4-positive,alpha-betaTcell', 'ectodermalcell', 'placentalbasalplate', 'thymus', 'stomachsmoothmuscle', 'esophagus', 'psoasmuscle', 'largeintestine', 'mesodermalcell', 'skeletalmuscletissue', 'spleen', 'middlefrontalarea46', 'ovary', 'iPSDF19.11', 'substantianigra', 'endocrinepancreas', 'T-cell', 'effectormemoryCD4-positive,alpha-betaTcell', 'HUES48', 'cingulategyrus', 'Bcell', 'subcutaneousabdominaladiposetissue', 'aorta', 'urinarybladder', 'lung', 'musclelayerofcolon', 'UCSF-4', 'heartleftventricle', 'neurosphere', 'naturalkillercell', 'mucosaofrectum', 'stomach', 'amnion', 'colonicmucosa',

In [5]:
# make output dir
new_dir ="/dors/capra_lab/users/fongsl/roadmap/hg38_%s" % now

new_dir_cmd = "mkdir %s" % new_dir
os.system(new_dir_cmd)
print(new_dir)

/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20


In [24]:
for tissue_id in matched_dataset:

    h3k27ac_val = h3k27ac_dict[tissue_id] # key for h3k27ac.bed filename
    #sort_27ac = "sortBed -i %s > sorted_%s " %(h3k27ac_val, h3k27ac_val)
    #print(sort_27ac)
    #os.system(sort_27ac)
    #rename_cmd ="mv sorted_%s %s"%(h3k27ac_val, h3k27ac_val)
    #os.system(rename_cmd)
    
    h3k4me3_val = h3k4me3_dict[tissue_id] # key for h3k4me3.bed filename
    #sort_me3 = "sortBed -i %s > sorted_%s " %(h3k4me3_val, h3k4me3_val)
    #print(sort_me3)
    #os.system(sort_me3)
    #rename_cmd ="mv sorted_%s %s"%(h3k4me3_val, h3k4me3_val)
    #os.system(rename_cmd)
    output_file = "Hsap_H3K27ac_plus_H3K4me3_minus_%s.bed" % (tissue_id)

    #STRINGENT enhancer definition. 180122 stringent analysis.
    os_cmd = "bedtools intersect -a %s -b %s -v -sorted > '%s/%s'" % (h3k27ac_val, h3k4me3_val, new_dir, output_file) 

    #RELAXED enhancer definition. 180123 relaxed analysis.
    #os_cmd = "bedtools subtract -a %s -b %s > '%s/%s'"% (h3k27ac_val, h3k4me3_val, new_dir, output_file) 

    print(os_cmd)

    os.system(os_cmd)



bedtools intersect -a HSap_ENCFF835UOA_EFO0003045_H9_H3K27ac.bed -b HSap_ENCFF485ZIC_EFO0003045_H9_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_EFO0003045.bed'
bedtools intersect -a HSap_ENCFF472NTY_UBERON0007106_chorionic_villus_H3K27ac.bed -b HSap_ENCFF816IVW_UBERON0007106_chorionic_villus_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_UBERON0007106.bed'
bedtools intersect -a HSap_ENCFF275ZWJ_UBERON0002107_liver_H3K27ac.bed -b HSap_ENCFF726WMX_UBERON0002107_liver_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_UBERON0002107.bed'
bedtools intersect -a HSap_ENCFF384FUV_CL0000897_CD4-positive,_alpha-beta_memory_T_cell_H3K27ac.bed -b HSap_ENCFF629PCT_CL0000897_CD4-positive,_alpha-beta_memory_T_cell_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4

bedtools intersect -a HSap_ENCFF749RHX_EFO0007096_iPS_DF_19.11_H3K27ac.bed -b HSap_ENCFF519CNY_EFO0007096_iPS_DF_19.11_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_EFO0007096.bed'
bedtools intersect -a HSap_ENCFF818ISZ_UBERON0002038_substantia_nigra_H3K27ac.bed -b HSap_ENCFF631ETT_UBERON0002038_substantia_nigra_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_UBERON0002038.bed'
bedtools intersect -a HSap_ENCFF459QWJ_UBERON0000016_endocrine_pancreas_H3K27ac.bed -b HSap_ENCFF729BGV_UBERON0000016_endocrine_pancreas_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_UBERON0000016.bed'
bedtools intersect -a HSap_ENCFF963JHV_CL0000084_T-cell_H3K27ac.bed -b HSap_ENCFF598DYJ_CL0000084_T-cell_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_CL000008

bedtools intersect -a HSap_ENCFF844RUD_CL0001059_common_myeloid_progenitor,_CD34-positive_H3K27ac.bed -b HSap_ENCFF655QJF_CL0001059_common_myeloid_progenitor,_CD34-positive_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_CL0001059.bed'
bedtools intersect -a HSap_ENCFF253UKG_UBERON0001873_caudate_nucleus_H3K27ac.bed -b HSap_ENCFF788TVU_UBERON0001873_caudate_nucleus_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_UBERON0001873.bed'
bedtools intersect -a HSap_ENCFF321NYQ_CL0000625_CD8-positive,_alpha-beta_T_cell_H3K27ac.bed -b HSap_ENCFF269CAD_CL0000625_CD8-positive,_alpha-beta_T_cell_H3K4me3.bed -v -sorted > '/dors/capra_lab/users/fongsl/roadmap/hg38_2018-06-20/Hsap_H3K27ac_plus_H3K4me3_minus_CL0000625.bed'
bedtools intersect -a HSap_ENCFF945WZJ_NTR0003079_fibroblast_of_breast_H3K27ac.bed -b HSap_ENCFF814LCB_NTR0003079_fibroblast_of_breast_H3K4me3.bed -v -sorted >

In [7]:
df = pandas.read_table("HSap_ENCFF835UOA_EFO0003045_H9_H3K27ac.bed", sep= '\t', header = -1)

In [14]:
df[3] = df[2].subtract(df[1])
print(df[3].max())

3567
